# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

Clase 10-11-20 1:00 aprox
Todos los graficos con titulos, escrbir bien etiquetas, usar bien colores(no como matlab), si pintas dos datos con el mismo color usar mapas
de colores con mas colores

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
#alt.data_transformers.enable('data_server')
alt.data_transformers.disable_max_rows()
#alt.themes.enable('opaque')

%matplotlib inline

**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [ ]:
start_date = pd.to_datetime("2020-04-13")

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)
confirmados.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo.tail(7)

## Ejercicio 1

(10 puntos)

Mostrar cantidad de fallecidos a la fecha por cada grupo etáreo.

In [ ]:
alt.Chart(fallecidos_etareo).mark_bar().encode(  
    x=alt.X('grupo_de_edad', axis=alt.Axis(title='Grupo Etario')),
    y=alt.Y('fallecidos', axis=alt.Axis(title='Fallecidos')),
    #color='grupo_de_edad:N',
    color=alt.Color('grupo_de_edad:N',legend=alt.Legend(title="Grupo Etareo"),scale=alt.Scale(scheme='brownbluegreen')),
    tooltip=['grupo_de_edad','fallecidos'],
    ).properties(
    width=600,
    height=500,
    title="FALLECIDOS POR GRUPO ETARIO A LA FECHA"
)

**Comentarios:** Al observar el gráfico nos damos cuenta que la mayor cantidad de fallecidos se encuentra en el grupo etario de los 70 a los 79 con 4339 fallecidos a 02 de noviembre. El segundo grupo con mayor cantidad de fallecidos es el de 80 a 89 años con 3991. Lo otro interesante es que el grupo de 0 a 39 años tiene una baja cantidad de fallecidos a pesar de ser el grupo con mayor rango (39 años y 4 veces mayor al resto) por lo que se puede inferir que el COVID-19 es menos agresivo con este grupo etario.

## Ejercicio 2

(10 puntos)

¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.

In [ ]:
alt.Chart(activos).mark_bar().encode(
    alt.X("comuna:O",axis=alt.Axis(title='Comunas de Chile')),
    y=alt.Y('poblacion',axis=alt.Axis(title='Población')),
    color=alt.Color('poblacion:Q',legend=alt.Legend(title="Población"),scale=alt.Scale(scheme='tealblues')),
    tooltip=['comuna','poblacion'],
).properties(
    width=5000,
    height=500,
    title="VARIABILIDAD DE POBLACION SEGUN COMUNAS DE CHILE"
)

**Comentarios:**  Se observa que la población por comuna de Chile tiene una gran variabilidad con un máximo de población para la comuna de Puente Alto de 645.909 habitantes y un mínimo de población para la comuna Antártica con 137 habitantes lo cual se entiende por lo inhóspito del clima. Lo otro que resalta que la mayoría de las comunas con mayor cantidad de habitantes están en la región Metropolitana. La comuna no perteneciente a la región metropolitana con mayor cantidad de habitantes es Antofagasta con 425.725 habitantes.

## Ejercicio 3

(10 puntos)

Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.

In [ ]:
fallecidos_etareo.head()

In [ ]:
f_etarios=alt.Chart(fallecidos_etareo).mark_line().encode(   
    x=alt.X('fecha', axis=alt.Axis(title='Fecha')),
    y=alt.Y('fallecidos', axis=alt.Axis(title='Cantidad de Fallecidos')),
    color=alt.Color('grupo_de_edad:N',legend=alt.Legend(title="Grupo Etareo")), 
    tooltip=["grupo_de_edad","fecha","fallecidos"]
).properties(
    width=1000, height=800
).interactive()

f_totales = alt.Chart(fallecidos_etareo).mark_line().encode(   
    x=alt.X('fecha', axis=alt.Axis(title='Fecha')),#'fecha',
    y=alt.Y('sum(fallecidos)', axis=alt.Axis(title='Cantidad de Fallecidos')),
    color=alt.value('blue'),
    tooltip=["fecha","sum(fallecidos)"]
).properties(
    width=1000,
    height=800,
    title="EVOLUCION DE FALLECIDOS A LA FECHA POR GRUPO ETAREO Y FALLECIDOS TOTALES (CURVA AZUL)"
).interactive()

f_etarios + f_totales

**Comentarios:** En este gráfico se puede observar la tendencia (o pendiente aproximada) de fallecidos por grupo etario es similar para los rangos de edad de 70 a 79, 80 a 89 y 60 a 69, con una pendiente mayor al resto de los grupos. Así también la tendencia o pendiente de evolución es similar para el rango de 50 a 59 y para mayores de 90 con una pendiente menor al rango anteriormente mencionado. Por último los grupos que tiene una pendiente similar entre si y menor al resto de los grupos son los de 0 a 39 y de 40 a 49. Con respecto a la curva de fallecidos totales podemos observar que las fechas con mayor aumento de fallecidos (pendiente) fueron los meses de junio y julio, lo cual también se refleja en las curvas para los grupos etarios. 

## Ejercicio 4

(10 puntos)

Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.

In [ ]:
line1 = alt.Chart(confirmados).mark_line(color='blue').encode(
    x=alt.X('fecha', axis=alt.Axis(title='Fecha')),#'fecha',
    y=alt.Y('sum(casos_confirmados)', axis=alt.Axis(title='Casos confirmados')),
    color=alt.value('#bcbd22'),
    tooltip=['fecha','sum(casos_confirmados)'],
).properties(
    width=400,
    height=300,
    title="SERIE DE TIEMPO CASOS CASOS CONFIRMADOS A LA FECHA"
)

line2 = alt.Chart(fallecidos).mark_line().encode(
    x=alt.X('fecha', axis=alt.Axis(title='Fecha')),#'fecha',
    y=alt.Y('sum(fallecidos)', axis=alt.Axis(title='Fallecidos')),
    color=alt.value('#ad494a'),
).properties(
    width=400,
    height=300,
    title="SERIE DE TIEMPO CASOS FALLECIDOS A LA FECHA"
)

line3 = alt.Chart(activos).mark_line().encode(
    x=alt.X('fecha', axis=alt.Axis(title='Fecha')),
    y=alt.Y('sum(casos_activos)', axis=alt.Axis(title='Casos Activos')),
    color=alt.value('#1f77b4')
).properties(
    width=400,
    height=300,
    title="SERIE DE TIEMPO CASOS ACTIVOS A LA FECHA"
)

(line1 | line2 | line3)

**Comentarios:** Las curvas de los gráficos de casos confirmados y fallecidos tienen un comportamiento similar, observando una mayor pendiente en los meses de junio y julio. Con respecto al gráfico de los casos activos podemos observar el pick de la curva a finales del mes de junio con lo cual, a pesar de que la curva no tenga un comportamiento similar a los otros dos gráficos si podría existir una relación entre la cantidad de casos confirmados y el mayor aumento (mayor pendiente) de casos confirmados y fallecidos en el periodo cercano a esta fecha.

## Ejercicio 5

(10 puntos)

Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.

In [ ]:
confir_tasa=confirmados.drop(['provincia_id','provincia','comuna_id','comuna'],axis=1).groupby(
        ["region", "fecha"]).apply(lambda df: df["casos_confirmados"].sum()/df["poblacion"].sum()*100000).reset_index().rename(columns={  0 : 'tasa'})
confir_tasa

In [ ]:
alt.Chart(confir_tasa).mark_line().encode(
    x=alt.X('fecha', axis=alt.Axis(title='Fecha')),
    y=alt.Y('tasa', axis=alt.Axis(title='Tasa de incidencia')),
    color=alt.Color('region:N',legend=alt.Legend(title="Regiones de Chile"),scale=alt.Scale(scheme='tableau20')),
    tooltip=['region','fecha','tasa'],
).properties(
    width=1200,
    height=600,
    title="SERIE DE TIEMPO DE TASA DE INCIDENCIA CASOS CONFIRMADOS POR REGION"
)

**Comentarios:** En este gráfico es difícil analizar el comportamiento de la tasa de incidencia de casos confirmados por región ya que las curvas se entrecruzan entre sí en varias fechas lo que no lo hace muy agradable a la vista. Sin embargo, de forma general podemos observar el drástico aumento de la tasas de incidencia de casos confirmados en la región de Magallanes desde septiembre a la fecha. También se puede observar que en varias regiones ha tendido a estabilizarse la tasa de incidencia.

## Ejercicio 6

(10 puntos)

¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.

In [ ]:
fecha1=confirmados.query( "fecha == '2020-04-13'" )

casos=alt.Chart(fecha1).mark_point().encode(
    x=alt.X('comuna', axis=alt.Axis(title='Comunas de Chile')),#'comuna',
    y=alt.Y('casos_confirmados', axis=alt.Axis(title='Casos Confirmados (Point)')),#'casos_confirmados',
    tooltip=['region','comuna','casos_confirmados'],
    size='poblacion:Q',
    color=alt.Color('region:N', scale=alt.Scale(scheme='tableau20'),legend=alt.Legend(title="Regiones de Chile"))
).properties(
    width=5000,
    height=600,
    title="CASOS CONFIRMADOS Y TASA DE INCIDENCIA POR COMUNA PARA EL 13 DE ABRIL DEL 2020"
).interactive()

tasa=alt.Chart(fecha1).mark_circle().encode(
    x=alt.X('comuna', axis=alt.Axis(title='Comunas de Chile')),#'comuna',
    y=alt.Y('tasa', axis=alt.Axis(title='Tasa de Incidencia (Circle)')),#'tasa',
    tooltip=['region','comuna','tasa'],
    size=alt.Color('poblacion:Q',legend=alt.Legend(title="Población")), #'poblacion:Q',
    color=alt.Color('region:N', scale=alt.Scale(scheme='tableau20'),legend=alt.Legend(title="Regiones de Chile"))
).properties(
    width=5000,
    height=600,
    title="CASOS CONFIRMADOS Y TASA DE INCIDENCIA POR COMUNA PARA EL 13 DE ABRIL DEL 2020"
).interactive()

tasa+casos

In [ ]:
fecha2=confirmados.query( "fecha == '2020-11-06'" )

casos=alt.Chart(fecha2).mark_point().encode(
    x=alt.X('comuna', axis=alt.Axis(title='Comunas de Chile')),
    y=alt.Y('casos_confirmados', axis=alt.Axis(title='Casos Confirmados (Point)')),
    tooltip=['region','comuna','casos_confirmados'],
    size=alt.value(100),
    color=alt.Color('region:N', scale=alt.Scale(scheme='tableau20'))
    
).properties(
    width=5000,
    height=600,
    title="CASOS CONFIRMADOS Y TASA DE INCIDENCIA POR COMUNA PARA EL 6 DE NOVIEMBRE DEL 2020"
)

tasa=alt.Chart(fecha2).mark_circle().encode(
    x=alt.X('comuna', axis=alt.Axis(title='Comunas de Chile')),#'comuna',
    y=alt.Y('tasa', axis=alt.Axis(title='Tasa de Incidencia (Circle)')),#'tasa',
    tooltip=['region','comuna','tasa'],
    size=alt.value(100),
    color=alt.Color('region:N', scale=alt.Scale(scheme='tableau20'),legend=alt.Legend(title="Regiones de Chile"))
).properties(
    width=5000,
    height=600,
    title="CASOS CONFIRMADOS Y TASA DE INCIDENCIA POR COMUNA PARA EL 6 DE NOVIEMBRE DEL 2020"
)
casos+tasa

**Comentarios:** Para la primera fecha se dejó el tamaño del circulo en función de la población, esto tiene sus ventajas y desventajas. Una ventaja es que es más fácil identificar las comunas con una mayor población y analizar la tasa y casos confirmados de estas. Esto puede servir ya que las comunas con mayor población tienen un efecto mayor en el análisis como país. Con respecto a las desventajas es que hay comunas con poca población que no se observan adecuadamente en el gráfico y a las cuales si hay que ponerle atención debido a que algunas tienen una alta tasa de incidencia. 
Con respecto al gráfico para la segunda fecha se definió un mismo tamaño de Marks para todos los datos y se estableció los círculos para la tasa de incidencia y los puntos (huecos) para los casos confirmados, lo que hace un poco más fácil la visualización.

## Ejercicio 7

(10 puntos)

1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico. 
2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

Entrega los pros y contras de cada uno de estos enfoques.

In [ ]:
line1 = alt.Chart(activos).mark_line().encode(
    x=alt.X('fecha', axis=alt.Axis(title='Fecha')),#'fecha',
    y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),#'casos_activos',
    color=alt.Color('comuna:N', scale=alt.Scale(scheme='spectral'),legend=alt.Legend(title="Comunas de Chile")),#'comuna',
    tooltip=['comuna','casos_activos']
).properties(
    width=1200,
    height=800,
    title="CASOS ACTIVOS POR COMUNAS DE CHILE"
)
line1 

In [ ]:
Arica_y_Parinacota=activos.query( "region == 'Arica y Parinacota'" )
r1 = alt.Chart(Arica_y_Parinacota).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
    width=500,height=500,title="CASOS ACTIVOS REGION ARICA Y PARINACOTA")

Tarapaca=activos.query( "region == 'Tarapaca'" )
r2 = alt.Chart(Tarapaca).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
    width=500,height=500,title="CASOS ACTIVOS REGION TARAPACA")

Antofagasta=activos.query( "region == 'Antofagasta'" )
r3 = alt.Chart(Antofagasta).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
   width=500,height=500,title="CASOS ACTIVOS REGION ANTOFAGASTA")

Atacama=activos.query( "region == 'Atacama'" )
r4 = alt.Chart(Atacama).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
    width=500,height=500,title="CASOS ACTIVOS REGION ATACAMA")

Coquimbo=activos.query( "region == 'Coquimbo'" )
r5 = alt.Chart(Coquimbo).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
    width=500,height=500,title="CASOS ACTIVOS REGION COQUIMBO")

Valparaiso=activos.query( "region == 'Valparaiso'" )
r6 = alt.Chart(Valparaiso).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
    width=500,height=500,title="CASOS ACTIVOS REGION VALPARAISO")

Metropolitana=activos.query( "region == 'Metropolitana'" )
r7 = alt.Chart(Metropolitana).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
    width=500,height=500,title="CASOS ACTIVOS REGION METROPOLITANA")

Del_Libertador=activos.query( "region == 'Del Libertador General Bernardo O’Higgins'" )
r8 = alt.Chart(Del_Libertador).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
    width=500,height=500,title="CASOS ACTIVOS REGION DEL LIBERTADOR BERNARDO O'HIGGINS")

Maule=activos.query( "region == 'Maule'" )
r9 = alt.Chart(Maule).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
    width=500,height=500,title="CASOS ACTIVOS REGION DEL MAULE")

Nuble=activos.query( "region == 'Nuble'" )
r10 = alt.Chart(Nuble).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
    width=500,height=500,title="CASOS ACTIVOS REGION DEL ÑUBLE")

Biobio=activos.query( "region == 'Biobio'" )
r11 = alt.Chart(Biobio).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
    width=500,height=500,title="CASOS ACTIVOS REGION DEL BIOBIO")

La_Araucania=activos.query( "region == 'La Araucania'" )
r12 = alt.Chart(La_Araucania).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
    width=500,height=500,title="CASOS ACTIVOS REGION DE LA ARAUCANIA")

Los_Rios=activos.query( "region == 'Los Rios'" )
r13 = alt.Chart(Los_Rios).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
    width=500,height=500,title="CASOS ACTIVOS REGION DE LOS RIOS")

Los_Lagos=activos.query( "region == 'Los Lagos'" )
r14 = alt.Chart(Los_Lagos).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
    width=500,height=500,title="CASOS ACTIVOS REGION DE LOS LAGOS")

Aysen=activos.query( "region == 'Aysen'" )
r15 = alt.Chart(Aysen).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
    width=500,height=500,title="CASOS ACTIVOS REGION DE AYSEN")

Magallanes_Antartica=activos.query( "region == 'Magallanes y la Antartica'" )
r16 = alt.Chart(Magallanes_Antartica).mark_line().encode(x=alt.X('fecha', axis=alt.Axis(title='Fecha')), y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos por comuna')),color='comuna',tooltip=['comuna','casos_activos']).properties(
    width=500,height=500,title="CASOS ACTIVOS REGION DE MAGALLANES Y LA ANTARTICA")



In [ ]:
(r1 | r2 | r3) 

In [ ]:
(r4 | r5 | r6) 

In [ ]:
(r7 | r8 | r9) 

In [ ]:
(r10 | r11 | r12)

In [ ]:
(r13 | r14 | r15) 

In [ ]:
r16

**Comentarios:** Gráfico con todas las comunas, Pros: Lo bueno de este gráfico es que se pueden comparar comunas de distintas regiones y observar las que tiene más casos activos en función de la fecha. Contras: Al estar todas las comunas de Chile se hace casi imposible la visualización de cada comuna, sobre todo las que tiene cantidad de casos similares.
Gráfico de las comunas por región, Pros: Tiene una mejor visualización de los casos activos por comuna de la región y es más fácil compararlas entre sí. Contras: No se puede comparar con comunas de otras regiones.


## Ejercicio 8

(10 puntos)

Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.

Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

In [ ]:
region_names = {
    "Araucanía": "La Araucanía",
    "La Araucania": "La Araucanía",
    "Aysén": "Aysén del General Carlos Ibáñez del Campo",
    "Aysen": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Magallanes y la Antartica": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
    "Del Libertador General Bernardo O’Higgins": "Libertador General Bernardo O'Higgins",
    "Biobio": "Biobío",
    "Los Rios": "Los Ríos",
    "Nuble": "Ñuble",
    "Valparaiso": "Valparaíso",
    "Tarapaca": "Tarapacá"  
}

In [ ]:
activos=activos.assign(region=lambda x: x["region"].map(region_names).fillna(x["region"]))
activos_region = (
   activos.groupby(["region"])
    .agg(
        prom_activos=("casos_activos", "mean"),        
    )
)

In [ ]:
confirmados=confirmados.assign(region=lambda x: x["region"].map(region_names).fillna(x["region"]))
max_confirm= (
   confirmados.groupby(["region"])
    .agg(
        max_conf=("casos_confirmados", "max"),
        #attack_max=("attack", "max")
    )
)

In [ ]:
fallecidos=fallecidos.assign(region=lambda x: x["region"].map(region_names).fillna(x["region"]))
max_fallec= (
   fallecidos.groupby(["region"])
    .agg(
        max_fallec=("fallecidos", "max"),
        )
).drop(['Total'],axis=0)

In [ ]:
grafico=activos_region.merge(
       max_confirm,
       how="left",
       on="region"
    ).merge(
        max_fallec,
        how="left",
        on="region"
   ).reset_index()
grafico

In [ ]:
grafico1=alt.Chart(grafico).mark_point().encode(
    x=alt.X('region', axis=alt.Axis(title='Región de Chile')),
    y=alt.Y('prom_activos', axis=alt.Axis(title='Prom. Activos (Punto)')),
    tooltip=['region','prom_activos'],
    size=alt.value(70),
    color=alt.Color('region:N', scale=alt.Scale(scheme='tableau20'),legend=alt.Legend(title="Regiones de Chile")),
)#.interactive()

grafico2=alt.Chart(grafico).mark_circle().encode(
    x=alt.X('region', axis=alt.Axis(title='Región de Chile')),#'region',
    y=alt.Y('max_conf', axis=alt.Axis(title='Max. Confirmados (Circulo Grande)')),
    tooltip=['region','max_conf'],
    size=alt.value(170),
    color=alt.Color('region:N', scale=alt.Scale(scheme='tableau20'),legend=alt.Legend(title="Regiones de Chile")),
)#.interactive()

grafico3=alt.Chart(grafico).mark_circle().encode(
    x=alt.X('region', axis=alt.Axis(title='Región de Chile')),
    y=alt.Y('max_fallec', axis=alt.Axis(title='Max. Fallecidos (Circulo pequeño)')),
    tooltip=['region','max_fallec'],
    size=alt.value(70),
    color=alt.Color('region:N', scale=alt.Scale(scheme='tableau20'),legend=alt.Legend(title="Regiones de Chile")),
)#.interactive()

(grafico1+grafico2+grafico3).properties(
    width=1000,
    height=700,
    title="PROMEDIO DE CASOS ACTIVOS, MAXIMO DE CASOS CONFIRMADOS Y MAXIMO DE FALLECIDOS POR REGION"
)


**Comentarios:** Este es un buen grafico para comparar entre regiones los promedios de casos activos, máximo de casos confirmados y fallecidos. Se identificó el promedio de casos activos con un punto hueco, el máximo de casos confirmados con un circulo grande y el máximo de fallecidos con un circulo pequeño para ayudar a una mejor visualización.

## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [ ]:
import geopandas as gpd   # instalar geopandas, agregarlo en enviroment
from pathlib import Path


In [ ]:
shp_filepath = Path().resolve().parent / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones.head()

Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [ ]:
alt.Chart(regiones).mark_geoshape().encode(
).properties(
    projection={'type': 'identity', 'reflectY': True}, # mapea en el mapa bidimensional
    width=250,
    height=600
)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olivdar utilizar el diccionario `region_names`).

In [ ]:
confirmados1=confirmados.groupby(["region", "fecha"]).agg(
        poblacion=("poblacion", "sum"),
        codigo_region=("region_id", "mean"),
        casos_confirmados=("casos_confirmados", "sum"),
        ).reset_index()
confirmados2=confirmados.drop(['provincia_id','provincia','comuna_id','comuna'],axis=1).groupby(
        ["region", "fecha"]).apply(lambda df: df["casos_confirmados"].sum()/df["poblacion"].sum()*100000).reset_index().rename(columns={  0 : 'tasa'})
confirmados_f=confirmados1.merge(confirmados2,how='right', on=['region', 'fecha']).replace({'poblacion': 82295.0}, 107297.0)

In [ ]:
activos=activos.assign(region=lambda x: x["region"].map(region_names).fillna(x["region"]))
activos1=activos.groupby(["region", "fecha"]).agg(
        poblacion=("poblacion", "sum"),
        codigo_region=("codigo_region", "max"),
        casos_activos=("casos_activos", "sum"),
        ).reset_index()

In [ ]:
conf_act=confirmados_f.merge(activos1,how='inner', on=['region', 'fecha','poblacion','codigo_region'])

In [ ]:
fallecidos=fallecidos.assign(region=lambda x: x["region"].map(region_names).fillna(x["region"]))
fallec= (
   fallecidos.groupby(["region","fecha"])
    .agg(
        fallecidos=("fallecidos", "max"),
        )
).drop(['Total'],axis=0).reset_index()

In [ ]:
conf_act_fall=conf_act.merge(fallec,how='inner', on=['region', 'fecha'])

In [ ]:
region_names1 = {
    "Región Metropolitana de Santiago": "Metropolitana de Santiago",
    "Región de Antofagasta": "Antofagasta",
    "Región de Arica y Parinacota": "Arica y Parinacota",
    "Región de Atacama": "Atacama",
    "Región de Aysén del Gral.Ibañez del Campo": "Aysén del General Carlos Ibáñez del Campo",
    "Región de Coquimbo": "Coquimbo",
    "Región de La Araucanía": "La Araucanía",
    "Región de Los Lagos": "Los Lagos",
    "Región de Los Ríos": "Los Ríos",
    "Región de Magallanes y Antártica Chilena": "Magallanes y de la Antártica Chilena",
    "Región de Tarapacá": "Tarapacá",
    "Región de Valparaíso": "Valparaíso",
    "Región de Ñuble": "Ñuble",
    "Región del Bío-Bío": "Biobío",
    "Región del Libertador Bernardo O'Higgins": "Libertador General Bernardo O'Higgins",
    "Región del Maule": "Maule"
}
    
regiones1=regiones.drop(['objectid','objectid','cir_sena','codregion','area_km','st_area_sh','st_length_'],axis=1).rename(
columns={'Region':'region'}).assign(region=lambda x: x["region"].map(region_names1).fillna(x["region"]))

In [ ]:
casos_geo = (conf_act_fall.merge(regiones1,how='inner', on=['region'])).sort_values(["codigo_region","fecha"])

Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [ ]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))
casos_geo#.dtypes

### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [ ]:
def covid_chile_chart(fecha, col):
    
    fecha = pd.to_datetime(fecha)
    data = casos_geo.loc[lambda x: x["fecha"] == fecha]#[# FIX ME #
    
    chart = alt.Chart(data).mark_geoshape().encode(
        color=col# FIX ME #
        ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=250,
        height=600
    )
    
    chart.display()
    return

Prueba con lo siguiente

In [ ]:
fecha = "2020-04-13"
col = "tasa"
covid_chile_chart(fecha, col)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [ ]:
from ipywidgets import interactive, fixed
import ipywidgets as widgets
from ipywidgets import interactive, interact
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

In [ ]:
col_widget = widgets.Dropdown(options=["poblacion","casos_confirmados","tasa","casos_activos","fallecidos"]
                              ,description='COLUMNA',continuous_update=False)                        

In [ ]:
fecha_widget = widgets.DatePicker(description='FECHA',continuous_update=False)

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [ ]:
covid_dashboard = interactive(covid_chile_chart,fecha=fecha_widget,col=col_widget)
covid_dashboard

**Comentarios:** Esta es una herramienta muy útil que ayuda a visualizar en un mapa de Chile como está distribuida la población por regiones. Así también, ayuda a identificar visualmente por fechas el comportamiento de las regiones en cuanto a casos confirmados, tasa de incidencia, casos activos y fallecidos. Esto sería muy útil para realizar estudios donde los resultados puedan ser apoyados con esta herramienta.